# Fuzzy Matching

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import dask.dataframe as dd
import os
import csv

os.chdir('/home/timothyelder/mag')

path = '/project/jevans/MAG_0802_2021_snap_shot/'

authors_df = dd.read_csv("/home/timothyelder/matches/matched_part_00.csv")

faculty_df = dd.read_csv("/home/timothyelder/mag/data/faculty_df_complete.csv")



In [3]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [4]:
# Load authors dataframe from MAG
authors_df = dd.read_csv(path + 'Authors.txt',
                                           sep="\t", header=None,
                                           error_bad_lines=False,
                                           quoting=csv.QUOTE_NONE,
                                           encoding='utf-8')

new_columns = ['AuthorId', 'Rank',
               'NormalizedName', 'DisplayName',
               'LastKnownAffiliationId', 'PaperCount',
               'PaperFamilyCount', 'CitationCount', 'CreatedDate']

authors_df = authors_df.rename(columns=dict(zip(authors_df.columns, new_columns)))

/home/timothyelder/.conda/envs/soc_of_soc/lib/python3.7/site-packages/dask/dataframe/io/csv.py:737: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  **kwargs,


In [ ]:
load faculty_df and normalize them then append 

In [ ]:
faculty_names = faculty_df.faculty_name.unique().compute()

NormalizedNames = authors_df.NormalizedName.unique().compute()

print(len(NormalizedNames))

NormalizedNames.append(faculty_names)

print(len(NormalizedNames))

/home/timothyelder/.conda/envs/soc_of_soc/lib/python3.7/site-packages/dask/dataframe/io/csv.py:134: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  path_info,
/home/timothyelder/.conda/envs/soc_of_soc/lib/python3.7/site-packages/dask/dataframe/io/csv.py:134: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  path_info,


In [18]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(NormalizedNames)

In [19]:
tf_idf_matrix

<85264586x6897527 sparse matrix of type '<class 'numpy.float64'>'
	with 1056209388 stored elements in Compressed Sparse Row format>

In [20]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.9)
t = time.time()-t1
print("SELFTIMED:", t)


In [ ]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [ ]:
import pandas as pd
matches_df = get_matches_df(matches, NormalizedNames, top=100000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

In [ ]:
len(NormalizedNames)

In [66]:
len(matches_df)

4751

In [45]:
authors_df.head()

,Unnamed: 0,best_match_score,__id_left,__id_right,faculty_name,network_name,AuthorId,NormalizedName
0,0,0.299692,0_left,1790018_right,"Siegenthaler, Jurg",jurg siegenthaler,328465342.0,thomas siegenthaler
1,16,0.069487,1_left,722178_right,"Clark, Leon E.",leon e. clark,229643539.0,o e sanchez leon
2,26,0.233462,2_left,1208458_right,"Young, Gloria (Gay) A.",gloria (gay) a. young,272560606.0,ko young aie
3,133,0.151519,3_left,540491_right,"Stone, Russell",russell stone,214803037.0,stone
4,195,0.050717,4_left,319507_right,"Dickerson, Bette J.",bette j. dickerson,146733665.0,bette j peltola


In [46]:
NormalizedNames

0       thomas siegenthaler
1          o e sanchez leon
2              ko young aie
3                     stone
4           bette j peltola
               ...         
5038            rene caquet
5039        lloyd d fricker
5040                 v zang
5041           alka dwevedi
5042         fredrik karell
Name: NormalizedName, Length: 5043, dtype: object